In [ ]:
import cv2 as cv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import math

In [ ]:
def process_dataset(dataset_dir):
    data = []
    labels = []
    paths = []

    for sub_folder in os.listdir(dataset_dir):
        sub_folder_files = os.listdir(os.path.join(dataset_dir, sub_folder))
        for i, filename in enumerate(sub_folder_files):
            img_path = os.path.join(dataset_dir, sub_folder, filename)
            img = cv.imread(img_path)
            data.append(img)
            labels.append(sub_folder)
            paths.append(img_path)

    labels = np.array(labels)
    paths = np.array(paths)

    return data, labels, paths

In [ ]:
def color_segmentation(images):
    segmented_images = []
    for image in images:
        hsv = cv.cvtColor(image, cv.COLOR_BGR2HSV)
        lower_green = np.array([36, 40, 40])
        upper_green = np.array([70, 255, 255])
        mask = cv.inRange(hsv, lower_green, upper_green)
        segmented_image = cv.bitwise_and(image, image, mask=mask)
        segmented_images.append(segmented_image)
    return segmented_images

In [ ]:
def edge_detection(images):
    edge_images = []
    for image in images:
        gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
        edges = cv.Canny(gray, 50, 255)
        edge_images.append(edges)
    return edge_images

In [ ]:
def enhance_image(images):
    enhanced_images = []
    for image in images:
        gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
        equalized_image = cv.equalizeHist(gray)
        enhanced_image = cv.cvtColor(equalized_image, cv.COLOR_GRAY2BGR)
        enhanced_images.append(enhanced_image)
    return enhanced_images

In [ ]:
def opening(images, kernel_size):
    opened_images = []
    kernel = np.ones((kernel_size, kernel_size), np.uint8)
    for image in images:
        eroded = cv.erode(image, kernel, iterations=1)
        opened = cv.dilate(eroded, kernel, iterations=1)
        opened_images.append(opened)
    return opened_images

In [ ]:
def normalize_images(images):
    normalized_images = []
    for img in images:
        normalized_img = cv.normalize(img, None, 0, 255, cv.NORM_MINMAX)
        normalized_images.append(normalized_img)
    return normalized_images

In [ ]:
dataset = "C:/Users/CATURWARGA COMPUTER/pyRPL/dataset"
data, labels, paths = process_dataset(dataset)

In [ ]:
segmented_images = color_segmentation(data)

enhanced_images = enhance_image(segmented_images)

normalized_images = normalize_images(enhanced_images)

opened_images = opening(normalized_images, kernel_size=2)

edge_images = edge_detection(opened_images)

In [ ]:
def detect_type(image):
    gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)

    edges = cv.Canny(gray, 50, 150)
    strong_edge_pixels = np.sum(edges > 200)
    weak_edge_pixels = np.sum(edges > 100)
    if strong_edge_pixels > weak_edge_pixels:
        forest_type = "lebat"
    elif weak_edge_pixels > strong_edge_pixels:
        forest_type = "setengahgundul"
    else:
        forest_type = "gundul"
    
    return forest_type

In [ ]:
def detect_forest_type(data, labels, paths):
    result_folder = "result"
    
    if not os.path.exists(result_folder):
        os.makedirs(result_folder)
    forest_types = ["gundul", "lebat", "setengahgundul"]
    
    for forest_type in forest_types:
        forest_type_folder = os.path.join(result_folder, forest_type)
        if not os.path.exists(forest_type_folder):
            os.makedirs(forest_type_folder)
    
    for i, img in enumerate(data):
        label = labels[i]
        img_path = paths[i]
        
        forest_type = detect_type(img)
        
        destination_folder = os.path.join(result_folder, forest_type)
        destination_path = os.path.join(destination_folder, os.path.basename(img_path))
        os.rename(img_path, destination_path)
        
        print(f"Moved {img_path} to {destination_path}")

In [ ]:
detect_forest_type(data, labels, paths)